FACE MASK DETECTION

In [1]:
#import required libraries
import tensorflow

In [2]:
#image generation and devide dataset into train and validation dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.20)

In [3]:
#train dataset
train_data=train_gen.flow_from_directory("mask_dataset",
                                          target_size=(224,224),batch_size=32,class_mode="binary",subset="training")

Found 643 images belonging to 2 classes.


In [4]:
#classes of train dataset
train_data.class_indices

{'with_mask': 0, 'without_mask': 1}

In [5]:
#for validation data
validation_data=train_gen.flow_from_directory("mask_dataset",
                                          target_size=(224,224),batch_size=32,class_mode="binary",subset="validation")

Found 160 images belonging to 2 classes.


In [6]:
#classes for validation dataset
validation_data.class_indices

{'with_mask': 0, 'without_mask': 1}

BUILD CNN ARCHITECTURE

In [7]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,BatchNormalization

In [8]:
#built architecture
model=Sequential()

model.add(Conv2D(filters=32,kernel_size=3,activation="relu",padding="same",input_shape=[224,224,3]))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units=32,activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(units=16,activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(units=1,activation="sigmoid"))

In [9]:
#summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 32)       128       
 ormalization)                                                   
                                                        

In [10]:
#compile the model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [11]:
#import callback for avoid overfitting
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor="accuracy",patience=10)

In [12]:
#fit the model
model.fit(train_data,validation_data=validation_data,callbacks=[es],epochs=20)

Epoch 1/20
21/21 [==============================] - 60s 3s/step - loss: 0.4673 - accuracy: 0.7823 - val_loss: 0.5708 - val_accuracy: 0.6062
Epoch 2/20
21/21 [==============================] - 45s 2s/step - loss: 0.2813 - accuracy: 0.9238 - val_loss: 0.6013 - val_accuracy: 0.5063
Epoch 3/20
21/21 [==============================] - 46s 2s/step - loss: 0.2736 - accuracy: 0.9129 - val_loss: 0.6481 - val_accuracy: 0.4938
Epoch 4/20
21/21 [==============================] - 46s 2s/step - loss: 0.2244 - accuracy: 0.9393 - val_loss: 0.5186 - val_accuracy: 0.7563
Epoch 5/20
21/21 [==============================] - 47s 2s/step - loss: 0.1978 - accuracy: 0.9440 - val_loss: 0.5557 - val_accuracy: 0.5500
Epoch 6/20
21/21 [==============================] - 46s 2s/step - loss: 0.1410 - accuracy: 0.9689 - val_loss: 0.5509 - val_accuracy: 0.5500
Epoch 7/20
21/21 [==============================] - 45s 2s/step - loss: 0.1360 - accuracy: 0.9658 - val_loss: 0.4310 - val_accuracy: 0.8750
Epoch 8/20
21/21 [==

In [13]:
#save both the model weights and model architecture in h5 file
model.save("mask_cnn.h5")

FACE MASK DETECTION FROM TRAINED MODEL(mask_cnn.h5)

In [14]:
#import trained CNN model for prediction
from tensorflow.keras.models import load_model

model=load_model("mask_cnn.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 32)       128       
 ormalization)                                                   
                                                        